# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>1. Initial Setting</b></div>

In [ ]:
import os 
os.environ['CUDA_IS_VISIBLE'] = '0,1'
import sys
import gc
import re
import ctypes
import random
from tqdm import tqdm
import polars as pl

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold


import torch

from transformers import AutoTokenizer, LongformerForSequenceClassification, DataCollatorWithPadding, AutoConfig

from transformers import Trainer, TrainingArguments
from datasets import Dataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class CFG:
    SEED = 2024
    VER = 1
    INFERENCE = True
    BASE_PATH = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/'
    preset = '/kaggle/input/aes2-longformer/DeBerta_BASE_v1'
    MAX_LEN = 1024
    TRAIN_BATCH = 4
    EVAL_BATCH = 4
    EPOCHS = 4

In [ ]:
Clean = True

def clean_memory():
    if Clean:
        ctypes.CDLL('libc.so.6').malloc_trim(0)
        gc.collect()
        
clean_memory()     

In [ ]:
def seed_everything():
    random.seed(CFG.SEED)
    os.environ['PYTHONHASHSEED'] = str(CFG.SEED)
    np.random.seed(CFG.SEED)
    torch.manual_seed(CFG.SEED)
    torch.cuda.manual_seed(CFG.SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>2. Road and Read Data</b></div>

In [ ]:
df_train = pd.read_csv(CFG.BASE_PATH + 'train.csv')
df_train = df_train.sort_values(by='essay_id')
df_train['label'] = df_train['score'] - 1

skf = StratifiedKFold(n_splits=5, random_state=CFG.SEED, shuffle=True)
for i, (_, val_index) in enumerate(skf.split(df_train,df_train['score'])):
    df_train.loc[val_index, 'fold'] = i

print('Shape of Train: ', df_train.shape)
print(display(df_train.head()))

In [ ]:
df_test = pd.read_csv(CFG.BASE_PATH + 'test.csv')
df_test = df_test.sort_values(by='essay_id')
print('Shape of Test: ', df_test.shape)
print(display(df_test.head()))

In [ ]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

In [ ]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html = re.compile(r'<.*?>')
    return html.sub(r'',x) # html -> ''

def dataPreprocessing(x):
    
    x = x.lower()
    # remove html
    x = removeHTML(x)
    
    #remove string with starting @
    x = re.sub("@\w+",'',x)
    
    # remove number
    x = re.sub("'\d+",'',x)
    x = re.sub("\d+",'',x)
    
    # remove url
    x = re.sub("http\w+",'',x)
    
    # Replace consecutive empty spaces with a single empty spaces
    x = re.sub(r'\s+', " ", x)
    # Replace consecutive commas and periods with a single comma and period
    x = expandContractions(x)
    x = re.sub(r'\.+', ".",x)
    x = re.sub(r'\,+', ",",x)
    x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
    # Remove empty character at the beginning and end
    x = x.strip()
    return x

In [ ]:
train = pl.from_pandas(df_train)
test = pl.from_pandas(df_test)

train = train.with_columns(
         pl.col('full_text').map_elements(lambda x: dataPreprocessing(x))
)
df_train = train.to_pandas()

test = test.with_columns(
         pl.col('full_text').map_elements(lambda x: dataPreprocessing(x))
)
df_test = test.to_pandas()

In [ ]:
df_train['label'] = df_train['label'].astype('float32')

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>3. Tokenizer</b></div>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.preset,  truncation=True, max_length=1500, clean_up_tokenization_spaces=False,)
print(tokenizer)

In [ ]:
def preprocess(sample):
    return tokenizer(sample['full_text'],)

In [ ]:
if CFG.INFERENCE is None:
    train = df_train[df_train['fold'] != 0]
    valid = df_train[df_train['fold'] == 0]

    dataset_v = Dataset.from_pandas(valid)
    dataset_t = Dataset.from_pandas(train)
    tokenized_dataset_v = dataset_v.map(preprocess, num_proc=2, 
                         remove_columns=['essay_id', 'full_text','score','fold'])
    tokenized_dataset_t = dataset_t.map(preprocess, num_proc=2, 
                        remove_columns=['essay_id', 'full_text','score','fold'])
else:
    dataset_train = Dataset.from_pandas(df_train)
    tokenized_dataset_train = dataset_train.map(preprocess, num_proc=2, 
                         remove_columns=['essay_id', 'full_text','score','fold'])

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>4. Training Model</b></div>

In [ ]:
training_args = TrainingArguments(
    output_dir = f'/output_v{CFG.VER}',
    per_device_train_batch_size=CFG.TRAIN_BATCH,
    per_device_eval_batch_size=CFG.EVAL_BATCH,
    num_train_epochs= CFG.EPOCHS,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    weight_decay=1e-3,
    fp16=True,
    learning_rate=1e-5,
    lr_scheduler_type='linear',
    optim='adamw_torch',
    metric_for_best_model='qwk',
    save_total_limit=1,
    report_to='none',
    gradient_checkpointing=True,
    gradient_accumulation_steps= 2
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    qwk = cohen_kappa_score(labels, predictions.clip(0,5).round(), weights='quadratic')
    results = {
        'qwk': qwk
    }
    return results

In [ ]:
model_config = AutoConfig.from_pretrained(CFG.preset)
model_config

In [ ]:
# Regression Config
model_config.attention_probs_dropout_prob = 0.0
model_config.hidden_dropout_prob = 0.0
# model_config.layer_norm_eps = 1e-8
model_config.num_labels = 1 

model =  LongformerForSequenceClassification.from_pretrained(CFG.preset, config=model_config)
    
trainer = Trainer(
       model = model,
       args = training_args,
       tokenizer = tokenizer,
       data_collator = DataCollatorWithPadding(tokenizer),
       compute_metrics=compute_metrics
    )
if CFG.INFERENCE is None:
    trainer.train()
else:
    None

In [ ]:
if CFG.INFERENCE is None:
    y_true = valid['score'].values
    predictions = trainer.predict(tokenized_dataset_v).predictions
    predictions = predictions + 1
    cm = confusion_matrix(y_true, predictions.clip(1,6).round(), labels=[x for x in range(1,7)])
    draw_cm = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=[x for x in range(1,7)])
    draw_cm.plot()
    plt.show()

    trainer.save_model(f'DeBerta_BASE_v{CFG.VER}')
else:
    None

In [ ]:
dataset_test = Dataset.from_pandas(df_test)
tokenized_test = dataset_test.map(preprocess, num_proc=3, 
                         remove_columns=['essay_id', 'full_text'])

In [ ]:
preds_test = trainer.predict(tokenized_test).predictions
preds_test = preds_test + 1 

In [ ]:
sub = pd.DataFrame({'essay_id': df_test.essay_id.values})
sub['score'] = preds_test.clip(1,6).round()
sub.to_csv('submission.csv',index=False)
print('Submission shape', sub.shape)
sub.head()